In [ ]:
%%capture
!pip install sdv
# !pip install numpy==1.20
# !pip install gdown
!pip install table_evaluator

In [ ]:
import pandas as pd
import random
from google.colab import drive
import os
drive.mount('/content/drive')
# os.chdir('/content/drive/My Drive/Pepsi Synthetic Data Use Case/Pepsico App/')
!ls



from sdv.metadata import MultiTableMetadata



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
drive  sample_data


# Train

## inputs for the app:
Collecting all app inputs for the Train section here:

In [ ]:
#inputs from the app:

hh_demographic_frac = 0.01
seed = random.seed(9001)

real_data_metadata_folder = "/content/drive/My Drive/Pepsi Synthetic Data Use Case/Pepsico App/real_data/"

model_save_folder = "/content/drive/My Drive/Pepsi Synthetic Data Use Case/Pepsico App/model/"

model_save_filename = f'SDVv1.0_Dunhumby_{hh_demographic_frac}.pkl'



## these next few lines will not run ofcourse because the data is being read in the next dabba. I just want to collect all the app vars in one place.
AGE_DESC_filter = hh_demographic_df['AGE_DESC'].unique().tolist()  
HOMEOWNER_DESC_filter	 = hh_demographic_df['HOMEOWNER_DESC'].unique().tolist()
HH_COMP_DESC_filter = hh_demographic_df['HH_COMP_DESC'].unique().tolist()



In [ ]:
# Read in the three tables, filter etc.
hh_demographic_df = pd.read_csv(os.path.join(real_data_metadata_folder,"hh_demographic.csv"))
hh_demographic_df = hh_demographic_df[hh_demographic_df['AGE_DESC'].isin(AGE_DESC_filter)&
                                      hh_demographic_df['HOMEOWNER_DESC'].isin(HOMEOWNER_DESC_filter)&
                                      hh_demographic_df['HH_COMP_DESC'].isin(HH_COMP_DESC_filter)] 
hh_demographic_df = hh_demographic_df.sample( frac = hh_demographic_frac,random_state=seed  )



transaction_df = pd.read_csv(os.path.join(real_data_metadata_folder,"transaction_data.csv"))
transaction_df = transaction_df[['household_key',	'BASKET_ID',	'PRODUCT_ID',	'QUANTITY',	'SALES_VALUE'	,'TRANS_TIME']]
transaction_df =  transaction_df[transaction_df['household_key'].isin(hh_demographic_df.household_key)]
# transaction_df = transaction_df.sample( frac = 0.1)  # thin this data for the climate and my sanity


product_df = pd.read_csv(os.path.join(real_data_metadata_folder,"product.csv"))
product_df = product_df[['PRODUCT_ID',	'DEPARTMENT',	'BRAND']]
product_df =  product_df[product_df['PRODUCT_ID'].isin(transaction_df.PRODUCT_ID)]



# 1. Collect all the tables in a dict
real_data = {}
real_data["product"] = product_df
real_data["transaction"] = transaction_df
real_data["hh_demographic"] = hh_demographic_df

for k in real_data:
  print(f'{k}, {real_data[k].shape}')

# read in the metadata
metadata = MultiTableMetadata.load_from_json(filepath=os.path.join(real_data_metadata_folder,'Dunhumby_Metadata_v1.json'))

product, (5380, 3)
transaction, (12043, 6)
hh_demographic, (8, 8)


## Model gets trained and saved here

In [ ]:
from sdv.multi_table import HMASynthesizer

# Step 1: Create the synthesizer
synthesizer = HMASynthesizer(metadata)

# Step 2: Train the synthesizer
synthesizer.fit(real_data)


synthesizer.save(
    filepath= os.path.join(model_save_folder,model_save_filename)
)


# Generate


## collecting all app inputs here:

In [ ]:
# all these are inputs coming fromt he app, values given here are for testing purposes.

model_path = os.path.join(model_save_folder,model_save_filename)
scale_value= 0.1
synth_data_folder = model_save_folder

## if real_data_metadata_folder was filled in the Train section (i.e. if model was trained) then get this from there,
## else ask for user input. 
real_data_metadata_folder = real_data_metadata_folder 

## Synthetic Data Generation happens here

In [ ]:
from sdv.multi_table import HMASynthesizer

synthesizer = HMASynthesizer.load(
    filepath= model_path
)

synthetic_data = synthesizer.sample(scale=scale_value)



/usr/local/lib/python3.9/dist-packages/scipy/stats/_continuous_distns.py:694: RuntimeWarning: divide by zero encountered in _beta_ppf
  return _boost._beta_ppf(q, a, b)
/usr/local/lib/python3.9/dist-packages/copulas/multivariate/gaussian.py:314: DeprecationWarning: `covariance` will be renamed to `correlation` in v0.4.0
  warnings.warn('`covariance` will be renamed to `correlation` in v0.4.0',
/usr/local/lib/python3.9/dist-packages/copulas/multivariate/gaussian.py:314: DeprecationWarning: `covariance` will be renamed to `correlation` in v0.4.0
  warnings.warn('`covariance` will be renamed to `correlation` in v0.4.0',
/usr/local/lib/python3.9/dist-packages/copulas/multivariate/gaussian.py:314: DeprecationWarning: `covariance` will be renamed to `correlation` in v0.4.0
  warnings.warn('`covariance` will be renamed to `correlation` in v0.4.0',
/usr/local/lib/python3.9/dist-packages/copulas/multivariate/gaussian.py:314: DeprecationWarning: `covariance` will be renamed to `correlation` in v

In [ ]:
# save
synthetic_data['product'].to_excel(os.path.join(synth_data_folder,'product_synth.xlsx'))
synthetic_data['transaction'].to_excel(os.path.join(synth_data_folder,'transaction_synth.xlsx'))
synthetic_data['hh_demographic'].to_excel(os.path.join(synth_data_folder,'hh_demographic_synth.xlsx'))

for k in synthetic_data:
  print(f'{k}  {real_data[k].shape}  {synthetic_data[k].shape}')



product  (5380, 3)  (538, 3)
transaction  (12043, 6)  (1451, 6)
hh_demographic  (8, 8)  (8, 8)


## The Fit Metrics  piece

In [ ]:
from sdv.evaluation.multi_table import evaluate_quality

quality_report = evaluate_quality(
    real_data=real_data,
    synthetic_data=synthetic_data,
    metadata=metadata)


# App output:
print(f'Overall Quality Score: {quality_report.get_score()}')
print(f'Overall Quality Score: {quality_report.get_properties()}')



Creating report: 100%|██████████| 5/5 [00:00<00:00,  5.02it/s]



Overall Quality Score: 52.48%

Properties:
Column Shapes: 76.51%
Column Pair Trends: 63.49%
Parent Child Relationships: 17.44%
Overall Quality Score: 0.5248029712014761
Overall Quality Score:                      Property     Score
0               Column Shapes  0.765073
1          Column Pair Trends  0.634893
2  Parent Child Relationships  0.174442


# Evaluate
Evaluation piece goes here. 

##  The inputs are as follows:


In [ ]:
# all these are inputs coming fromt he app, values given here are for testing purposes.
table_to_display = 'hh_demographic'
variable_to_display  = 'AGE_DESC'
# table_to_display = 'transaction'
# variable_to_display  = 'SALES_VALUE'
## if real_data_metadata_folder was filled in the Train section (i.e. if model was trained) then get this from there,
## else ask for user input. 
real_data_metadata_folder = real_data_metadata_folder 

## if synth_data_folder was filled in the Train section (i.e. if synth data was generated) then get this from there,
## else ask for user input. 
synth_data_folder = synth_data_folder

In [ ]:
from sdv.evaluation.multi_table import get_column_plot

fig = get_column_plot(
    real_data=real_data,
    synthetic_data=synthetic_data,
    table_name= table_to_display,
    column_name= variable_to_display,
    metadata=metadata
)

# App output  
fig.show()